<a href="https://colab.research.google.com/github/hussain0048/Machine-Learning/blob/master/Anomaly_Detection_using_Using_Gaussian_Mixture_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Loading**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#**Import libaray**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from scipy.io import loadmat
%matplotlib inline

# **Dataset**

In [ ]:
# Number of samples per component
n_samples = 500

In [4]:
# Generate random sample, two components
import numpy as np
# Generate random sample, two components
np.random.seed(0)
C = np.array([[0., -0.1], [1.7, .4]])
C2 = np.array([[1., -0.1], [2.7, .2]])
#X = np.r_[np.dot(np.random.randn(n_samples, 2), C)]
          #.7 * np.random.randn(n_samples, 2) + np.array([-6, 3])]
X = np.r_[np.dot(np.random.randn(n_samples, 2), C),np.dot(np.random.randn(n_samples, 2), C2)]

# **Data Ploting**

In [7]:
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
X[-5:] = [[4,-1],[4.1,-1.1],[3.9,-1],[4.0,-1.2],[4.0,-1.3]]

In [ ]:
plt.scatter(X[:,0], X[:,1],s=5)

# **Model development**

In [11]:
from sklearn.mixture import GaussianMixture

In [12]:
gmm = GaussianMixture(n_components=3)


In [13]:
gmm.fit(X)

GaussianMixture(n_components=3)

In [14]:
pred = gmm.predict(X)

In [15]:
pred[:50]

array([1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2])

In [ ]:
plt.scatter(X[:,0], X[:,1],s=10,c=pred)

# **References**

[1-Anomaly Detection.ipynb](https://github.com/edyoda/data-science-complete-tutorial/blob/master/13.%20Anomaly%20Detection.ipynb)